In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf

In [2]:
data = pd.read_csv('data/diabetes_012_health_indicators_BRFSS2021.csv')

# Display first few rows of the dataset
print(data.head())

# Check for missing values
print(data.isnull().sum())

   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0           0.0       0       1.0          1  15.0     1.0     0.0   
1           2.0       1       0.0          1  28.0     0.0     0.0   
2           2.0       1       1.0          1  33.0     0.0     0.0   
3           2.0       0       1.0          1  29.0     0.0     1.0   
4           0.0       0       0.0          1  24.0     1.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0             0       1  ...              1   
1                   1.0             0       1  ...              1   
2                   0.0             1       1  ...              1   
3                   1.0             1       1  ...              1   
4                   0.0             0       0  ...              1   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex  Age  Education  \
0          0.0      5.0      10.0      20.0       0.0    0   11        4.0   
1       

In [4]:
# tach cot Benh ly tieu duong ra khoi df
X = data.drop('Diabetes_012', axis=1)
y = data['Diabetes_012']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# toi uu hoa LSTM
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# chuyen thanh danh danh muc
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')  # Assuming 2 classes for diabetes: 0 or 1
])

# model tuan tu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
4728/4728 [==============================] - 18s 3ms/step - loss: 0.4419 - accuracy: 0.8367 - val_loss: 0.4285 - val_accuracy: 0.8400
Epoch 2/50
4728/4728 [==============================] - 14s 3ms/step - loss: 0.4327 - accuracy: 0.8373 - val_loss: 0.4299 - val_accuracy: 0.8372
Epoch 3/50
4728/4728 [==============================] - 14s 3ms/step - loss: 0.4312 - accuracy: 0.8384 - val_loss: 0.4266 - val_accuracy: 0.8395
Epoch 4/50
4728/4728 [==============================] - 14s 3ms/step - loss: 0.4301 - accuracy: 0.8384 - val_loss: 0.4262 - val_accuracy: 0.8398
Epoch 5/50
4728/4728 [==============================] - 14s 3ms/step - loss: 0.4298 - accuracy: 0.8379 - val_loss: 0.4260 - val_accuracy: 0.8410
Epoch 6/50
4728/4728 [==============================] - 14s 3ms/step - loss: 0.4291 - accuracy: 0.8383 - val_loss: 0.4264 - val_accuracy: 0.8397
Epoch 7/50
3011/4728 [==================>...........] - ETA: 4s - loss: 0.4255 - accuracy: 0.8395

KeyboardInterrupt: 

In [6]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

# Predict from trained model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Report
print(classification_report(y_true, y_pred_classes))

1478/1478 [==============================] - 2s 1ms/step - loss: 0.4179 - accuracy: 0.8411
Test Accuracy: 84.11%
1478/1478 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

           0       0.85      0.98      0.91     39508
           1       0.00      0.00      0.00      1079
           2       0.56      0.14      0.22      6689

    accuracy                           0.84     47276
   macro avg       0.47      0.37      0.38     47276
weighted avg       0.79      0.84      0.79     47276



c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
